In [1]:
# !pip install split-folders
!pip install scikeras

#import mediapipe as mp
import time
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import tensorflow as tf
import math
from tensorflow.keras.layers import Bidirectional, LSTM, Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, Reshape, GlobalAveragePooling1D, Add, Reshape
from tensorflow.keras.models import Model, Sequential
from sklearn.metrics import mean_absolute_error
from keras.layers import Input, MultiHeadAttention, Dense, Dropout, LayerNormalization, Add, Reshape, Flatten

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Reshape, Bidirectional, Input, MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D, Dropout, Add, Flatten
from tensorflow.keras.utils import plot_model
import itertools
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import LSTM, Bidirectional, Dense, Reshape
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from scikeras.wrappers import KerasRegressor

from collections import Counter
from tensorflow.keras.models import load_model,Sequential
from tensorflow.keras.layers import LSTM, Dense, Reshape
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle
from tqdm import tqdm


In [2]:
# import splitfolders
# splitfolders.ratio("D:\\Stroke Data", output="D:\\Stroke Data_Splitted\\",seed=1337, ratio=(.7, .3), group_prefix=None, move=False)

In [3]:
'''
def draw_and_save_model(model, filename):
    # Plot the model architecture and save as an image
    plot_model(model, to_file=filename, show_shapes=True, show_layer_names=True)
    img = plt.imread(filename)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.show()
'''
# Function to plot and save BERT model architecture
def draw_and_save_model(model, filename):
    # Plot the model architecture with increased DPI for better image quality and readability
    plot_model(model, to_file=filename, show_shapes=True, show_layer_names=True,
               dpi=150)  # Increased DPI for better resolution
    img = plt.imread(filename)
    plt.figure(figsize=(15, 15))  # Larger figure for better visibility
    plt.imshow(img)
    plt.axis('off')
    plt.show()

def lstm_model(NextFrames,data,X_train, y_train, X_test, y_test,train_size, outputunit, learning_rate, batch_size):
  # Build the LSTM model
  model = Sequential()
  model.add(LSTM(outputunit, activation='relu', return_sequences=True, input_shape=(sequence_length, data.shape[1])))
  #model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(sequence_length, data.shape[1])))
  model.add(LSTM(outputunit, activation='relu'))
  #model.add(LSTM(128, activation='relu'))
  model.add(Dense(NextFrames * data.shape[1]))  # Predicting 3 frames (3 * 132 features)
  model.add(Reshape((NextFrames, data.shape[1])))  # Reshape output to (3, 132)
  # Compile the model
    # Compile with adjustable learning rate
  optimizer = Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer, loss='mean_squared_error')
  #optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  #model.compile(optimizer=optimizer, loss='mse')
  #model.compile(optimizer='adam', loss='mean_squared_error')

  # Train the model
  model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_split=0.2, verbose=0)
  # Predict the next 3 frames
  print(f"X_test[:1].shape {X_test[:1].shape}")
  predictions_scaled = model.predict(X_test[:1])  # Predict only the first set of test data (next 3 frames)
  # Inverse transform the predictions to the original scale
  predictions = scaler.inverse_transform(predictions_scaled.reshape(NextFrames, -1))
  # Get the actual next 5 frames (841, 842, 843)
  actual = data[train_size:train_size+NextFrames]
  #print(f"y_test shape: {y_test.shape}, actual.shape: {actual.shape}")
  # Evaluate performance using Mean Absolute Error (MAE)
  mae = mean_absolute_error(actual, predictions)
  return model,actual,predictions, mae

def Bilstm_model(NextFrames,data,X_train, y_train, X_test, y_test,train_size, outputunit, learning_rate, batch_size):
  #sequence_length = X_train.shape[1]  # Define based on your data
  #n_features = X_train.shape[2]       # Define based on your data
  model = Sequential()
  model.add(Bidirectional(LSTM(outputunit, activation='relu', return_sequences=True), input_shape=(sequence_length, data.shape[1])))
  #model.add(Bidirectional(LSTM(128, activation='relu', return_sequences=True), input_shape=(sequence_length, data.shape[1])))
  model.add(Bidirectional(LSTM(outputunit, activation='relu')))
  #model.add(Bidirectional(LSTM(128, activation='relu')))
  model.add(Dense(NextFrames * data.shape[1] ))  # Predicting next frames with shape (NextFrames * n_features)
  model.add(Reshape((NextFrames, data.shape[1])))  # Reshape to (NextFrames, n_features)

  optimizer = Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer, loss='mean_squared_error')
  #optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  #model.compile(optimizer=optimizer, loss='mse')

  #model.compile(optimizer='adam', loss='mse')

  # Train the model
  model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_split=0.2, verbose=0)
  # Predict the next 3 frames
  predictions_scaled = model.predict(X_test[:1])  # Predict only the first set of test data (next 3 frames)

  # Inverse transform the predictions to the original scale
  predictions = scaler.inverse_transform(predictions_scaled.reshape(NextFrames, -1))

  # Get the actual next 3 frames (841, 842, 843)
  actual = data[train_size:train_size+NextFrames]

  # Evaluate performance using Mean Absolute Error (MAE)
  mae = mean_absolute_error(actual, predictions)
  return model, actual,predictions, mae

################################

def transformer_model(NextFrames, sequence_length, n_features, X_train, y_train, X_test, y_test,train_size, outputunit, MultiHead,dropout):
    input_layer = Input(shape=(sequence_length, n_features))

    # Multi-Head Attention Layer
    attention_output = MultiHeadAttention(num_heads=MultiHead, key_dim=n_features)(input_layer, input_layer)
                     #MultiHeadAttention(num_heads=4, key_dim=n_features)(input_layer, input_layer)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Fully Connected Layers
    transformer_output = Dense(outputunit, activation='relu')(attention_output)
                       #Dense(64, activation='relu')(attention_output)
    transformer_output = Dropout(dropout)(transformer_output)
                        #Dropout(0.2)(transformer_output)

    # Pooling to reduce dimensions to match NextFrames
    transformer_output = GlobalAveragePooling1D()(transformer_output)
    # Predict the next frames (NextFrames * n_features)
    output_layer = Dense(NextFrames * n_features)(transformer_output)
    output_layer = Reshape((NextFrames, n_features))(output_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=0)
    # Predict for the entire X_test set
    #predictions = model.predict(X_test)
      # Predict the next 3 frames
    print(f"X_test[:1].shape {X_test[:1].shape}")

    predictions_scaled = model.predict(X_test[:1])  # Predict only the first set of test data (next 3 frames)
    predictions = scaler.inverse_transform(predictions_scaled.reshape(NextFrames, -1))

    actual = data[train_size:train_size+NextFrames]
    print(f"actual.shape = {actual.shape}, prediction.shape = {predictions.shape}")
    #ytest=y_test.reshape(-1, n_features)
    #predic=predictions.reshape(-1, n_features)

    #print(f"y_test.reshape(-1, n_features)= {ytest.shape}, predictions.reshape(-1, n_features) = {predic.shape}, actual shape: {actual.shape}")

    # Evaluate performance using Mean Absolute Error (MAE)
    #mae = mean_absolute_error(y_test.reshape(-1, n_features), predictions.reshape(-1, n_features))
    mae = mean_absolute_error(actual, predictions)
    print(f"ourput unit: {outputunit} dropout= {dropout}, MultiHead = {MultiHead}, mae: {mae}")
    return model, actual, predictions, mae

###########################################
def bert_model(NextFrames, sequence_length, n_features, X_train, y_train, X_test, y_test, train_size, MultiHead, dropout, num_transformer_blocks):
    input_layer = Input(shape=(sequence_length, n_features))

    # Transformer blocks emulating BERT layers
    x = input_layer
    for _ in range(num_transformer_blocks):
        # Multi-Head Attention layer with normalization and dropout
        attention_output = MultiHeadAttention(num_heads=MultiHead, key_dim=n_features)(x, x)
                          #MultiHeadAttention(num_heads=4, key_dim=n_features)(x, x)
        attention_output = Dropout(dropout)(attention_output)
        attention_output = Add()([x, attention_output])  # Residual connection
        attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

        # Feed-forward network
        ffn_output = Dense(n_features, activation='relu')(attention_output)  # Match n_features to maintain shape
        ffn_output = Dropout(dropout)(ffn_output)  #Dropout(0.1)(ffn_output)
        x = Add()([attention_output, ffn_output])  # Another residual connection
        x = LayerNormalization(epsilon=1e-6)(x)

    # Flatten and add dense layer to ensure correct output shape
    x = Flatten()(x)
    output_layer = Dense(NextFrames * n_features)(x)
    output_layer = Reshape((NextFrames, n_features))(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=0)

    # Predict for the entire X_test set
    predictions_scaled = model.predict(X_test[:1])  # Predict only the first set of test data (next frames)

    # Assuming `scaler` and `data` are defined, adapt as needed
    predictions = scaler.inverse_transform(predictions_scaled.reshape(NextFrames, -1))
    actual = data[train_size:train_size + NextFrames]

    mae = mean_absolute_error(actual, predictions)
    print(f"MultiHead: {MultiHead}, dropout:{dropout},  MAE: {mae}")
    return model, actual, predictions, mae


In [4]:
def model_evaluation(NextFrames, y_test,y_pred, ModelName, stroke, predictedFrameNumber):

  from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

  # Initialize lists to store metrics for each frame
  mae_scores = []
  mse_scores = []
  r2_scores = []

  # Loop through each of the 5 frames
  for i in range(NextFrames):
      # Get the actual and predicted values for the i-th frame
      actual_frame = y_test[ i, :]    # Shape: (num_samples, n_features)
      predicted_frame = y_pred[ i, :] # Shape: (num_samples, n_features)

      # Calculate MAE, MSE, and R² for the i-th frame
      mae = mean_absolute_error(actual_frame, predicted_frame)
      mse = mean_squared_error(actual_frame, predicted_frame)
      r2 = r2_score(actual_frame, predicted_frame)

      # Append the scores to the lists
      mae_scores.append(mae)
      mse_scores.append(mse)
      r2_scores.append(r2)

      #plt.subplot(3, 1, i+1)
      #plt.plot(, label='Predicted Frame {}'.format(train_size + i + 1))
      predictions_rescaled=predicted_frame
      test_data_scaled = actual_frame
      # Print predicted vs actual frames for comparison
      print("Predicted frames:")
      #print(predictions_rescaled)
      print("Actual frames:")
      #print(test_data_scaled)
      pred_actual_frame(Posdataset, predictions_rescaled, test_data_scaled)

      preframes = predictions_rescaled
      # Reshape frames to 2D (if it is a 3D array)
      preframes = preframes.reshape(1, 132)
      dataset = Posdataset.drop(columns=['Unnamed: 0', 'Label'], axis=1)
      # Assuming 'newdataset' is your DataFrame
      columns_list = dataset.columns.tolist()

      # Now create the DataFrame
      PredictedFrame = pd.DataFrame(preframes, columns=columns_list)

      orgframes = test_data_scaled
      orgframes = orgframes.reshape(1, 132)
      OrignalFrame = pd.DataFrame(orgframes, columns=columns_list)

      #VisualEvaluation_3Actual_Predicted_Frame(PredictedFrame, OrignalFrame)
      predictedFrameNumber=predictedFrameNumber+i
      mse,mse,r2=qualitative_quantitative_diff(PredictedFrame, OrignalFrame, ModelName, stroke,predictedFrameNumber)
      # Display the scores for each frame
  print(f" { ModelName } Model Evaluation for Batsman's Stroke {stroke}: ")
  return mse,mse,r2, ModelName, stroke
  '''
  for i in range(NextFrames):
      print(f"Frame {i+1}: MAE = {mae_scores[i]}, MSE = {mse_scores[i]}, R² = {r2_scores[i]}")
  # Average scores across all frames
  average_mae = sum(mae_scores) / NextFrames
  average_mse = sum(mse_scores) / NextFrames
  average_r2 = sum(r2_scores) / NextFrames

  print(f"Over All {ModelName} Average MAE: {average_mae}")
  print(f"Over All {ModelName}  Average MSE: {average_mse}")
  print(f"Over All {ModelName}  Average R²: {average_r2}")
  '''
#############################################

import pandas as pd
import numpy as np
import plotly.graph_objects as go

def extract_coordinates(df):
    joints = 33  # Number of joints
    x, y, z = [], [], []

    for i in range(joints):
        x.append(df[f'F{i}_x'].values[0])  # Take the first row's x value
        y.append(df[f'F{i}_y'].values[0])  # Take the first row's y value
        z.append(df[f'F{i}_z'].values[0])  # Take the first row's z value

    return np.array(x), np.array(y), np.array(z)

# Function to plot the skeleton for a single pose in 3D
def plot_skeleton(x, y, z, connections, color='blue'):
    lines = []
    for joint1, joint2 in connections:
        lines.append(go.Scatter3d(
            x=[x[joint1], x[joint2]],
            y=[y[joint1], y[joint2]],
            z=[z[joint1], z[joint2]],
            mode='lines',
            line=dict(color=color, width=5)
        ))
    return lines

def VisualEvaluation_3Actual_Predicted_Frame(PredictedFrame, OrignalFrame):
  # Load DataFrame with 33 joints (columns: 'x', 'y', 'z', 'visibility')
  df1 = PredictedFrame  # Replace with your actual data
  df2 = OrignalFrame  # Replace with your actual data

  # Extract coordinates for both poses
  x1, y1, z1 = extract_coordinates(df1)
  x2, y2, z2 = extract_coordinates(df2)

  # Define connections between joints based on MediaPipe PoseNet model
  POSE_CONNECTIONS = [
      (0, 1), (1, 2), (2, 3), (3, 7),  # Right arm
      (0, 4), (4, 5), (5, 6), (6, 8),  # Left arm
      (9, 10), (11, 12),               # Shoulders
      (11, 13), (13, 15), (15, 17),    # Left leg
      (12, 14), (14, 16), (16, 18),    # Right leg
      (11, 23), (12, 24),              # Hips to shoulders
      (23, 24), (23, 25), (24, 26),    # Hips and upper legs
      (25, 27), (27, 29), (29, 31),    # Left lower leg
      (26, 28), (28, 30), (30, 32)     # Right lower leg
  ]
  '''
  # Create the 3D plot with both poses
  fig = go.Figure()

  # Add skeleton for both poses (in red and blue)
  fig.add_traces(plot_skeleton(x1, y1, z1, POSE_CONNECTIONS, color='red'))
  fig.add_traces(plot_skeleton(x2, y2, z2, POSE_CONNECTIONS, color='blue'))

  # Set the layout for the 3D plot with increased size
  fig.update_layout(
      title='3D Pose Skeleton Visualization',
      scene=dict(
          xaxis=dict(title='X-axis'),
          yaxis=dict(title='Y-axis'),
          zaxis=dict(title='Z-axis', range=[-1, 1]),  # Adjust range as needed
          aspectmode='cube'
      ),
      width=1200,  # Increased width
      height=800,  # Increased height
      showlegend=False
  )

  # Display the plot
  fig.show()
  '''
#####################################################################

def qualitative_quantitative_diff(predictions_rescaled, y_test_rescaled, model_name, stroke, predictedFrameNumber):
  import numpy as np
  import matplotlib.pyplot as plt
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

  # Assuming predictions_rescaled and y_test_rescaled are numpy arrays
  # Reshape the arrays to be one-dimensional for better comparison
  predictions_rescaled_flat = np.ravel(predictions_rescaled)
  y_test_rescaled_flat = np.ravel(y_test_rescaled)

  # Ensure the two arrays are of the same length by truncating if necessary
  min_length = min(len(predictions_rescaled_flat), len(y_test_rescaled_flat))
  predictions_rescaled_flat = predictions_rescaled_flat[:min_length]
  y_test_rescaled_flat = y_test_rescaled_flat[:min_length]

  # Define the x-axis labels
  x_labels = ["Nose_x", "Nose_y", "Nose_z", " ", "LEyeInner_x", "LEyeInner_y", "LEyeInner_z", " ",
              "LEye_x", "LEye_y", "LEye_z", " ", "LEyeOuter_x", "LEyeOuter_y", "LEyeOuter_z", " ",
              "REyeInner_x", "REyeInner_y", "REyeInner_z", " ", "REye_x", "REye_y", "REye_z", " ",
              "REyeOuter_x", "REyeOuter_y", "REyeOuter_z", " ", "LEar_x", "LEar_y", "LEar_z", " ",
              "REar_x", "REar_y", "REar_z", " ", "LMouth_x", "LMouth_y", "LMouth_z", " ",
              "RMouth_x", "RMouth_y", "RMouth_z", " ", "LShoulder_x", "LShoulder_y", "LShoulder_z", " ",
              "RShoulder_x", "RShoulder_y", "RShoulder_z", " ", "LElbow_x", "LElbow_y", "LElbow_z", " ",
              "RElbow_x", "RElbow_y", "RElbow_z", " ", "LWrist_x", "LWrist_y", "LWrist_z", " ",
              "RWrist_x", "RWrist_y", "RWrist_z", " ", "LPinky_x", "LPinky_y", "LPinky_z", " ",
              "RPinky_x", "RPinky_y", "RPinky_z", " ",    "LIndex_x", "LIndex_y", "LIndex_z", " ",
              "RIndex_x", "RIndex_y", "RIndex_z", " ",    "LThumb_x", "LThumb _y", "LThumb _z", " ",
              "RThumb_x", "RThumb _y", "RThumb _z", " ",    "LHip_x", "LHip_y", "LHip_z", " ",
              "RHip_x", "RHip_y", "RHip_z", " ", "LKnee_x", "LKnee_y", "LKnee_z", " ",
              "RKnee_x", "RKnee_y", "RKnee_z", " ", "LAnkle_x", "LAnkle_y", "LAnkle_z", " ",
              "RAnkle_x", "RAnkle_y", "RAnkle_z", " ", "LHeel_x", "LHeel_y", "LHeel_z", " ",
              "RHeel_x", "RHeel_y", "RHeel_z", " ","LFootIdx_x", "LFootIdx_y", "LFootIdx_z", " ",
              "RFootIdx_x", "RFootIdx_y", "RFootIdx_z", " ",
           ]


  # Quantitative metrics
  mse = mean_squared_error(y_test_rescaled_flat, predictions_rescaled_flat)
  mae = mean_absolute_error(y_test_rescaled_flat, predictions_rescaled_flat)
  r2 = r2_score(y_test_rescaled_flat, predictions_rescaled_flat)
  print(f"Mean Squared Error (MSE): {mse}")
  print(f"Mean Absolute Error (MAE): {mae}")
  print(f"R-squared (R²): {r2}")

  # Visual comparison: Plot actual vs. predicted values
  plt.figure(figsize=(15, 8))
  plt.plot(y_test_rescaled_flat, label='Actual Values', color='blue', marker='o')
  plt.plot(predictions_rescaled_flat, label='Predicted Values', color='red',  linestyle='--', marker='x')
  plt.title('Comparison of Actual and ' + model_name + '-Predicted Body Points at Frame '+ str(predictedFrameNumber) +' for ' + stroke + ' Motion')

  plt.xlabel('Data Points')
  plt.xticks(ticks=np.arange(len(x_labels)), labels=x_labels, rotation=90)
  plt.ylabel('Rescaled Values')
  plt.legend()
  plt.grid(True)
  plt.tight_layout()
  plt.show()

  return mse,mse,r2

###############################

def All_qualitative_quantitative_diff0(
        actual, LSTMpredictions, BiLSTMpredictions, TransformerPredictions, BERTpredictions, stroke):
    import numpy as np
    import matplotlib.pyplot as plt

    # Flatten predictions and actual values to ensure they are one-dimensional arrays
    LSTMpredictions_flat = np.ravel(LSTMpredictions)
    BiLSTMpredictions_flat = np.ravel(BiLSTMpredictions)
    TransformerPredictions_flat = np.ravel(TransformerPredictions)
    BERTpredictions_flat = np.ravel(BERTpredictions)
    actual_flat = np.ravel(actual)

    # Define the y-axis labels (132 keypoints as given in the original code)
    y_labels = [
        "Nose_x", "Nose_y", "Nose_z", " ", "LEyeInner_x", "LEyeInner_y", "LEyeInner_z", " ",
        "LEye_x", "LEye_y", "LEye_z", " ", "LEyeOuter_x", "LEyeOuter_y", "LEyeOuter_z", " ",
        "REyeInner_x", "REyeInner_y", "REyeInner_z", " ", "REye_x", "REye_y", "REye_z", " ",
        "REyeOuter_x", "REyeOuter_y", "REyeOuter_z", " ", "LEar_x", "LEar_y", "LEar_z", " ",
        "REar_x", "REar_y", "REar_z", " ", "LMouth_x", "LMouth_y", "LMouth_z", " ",
        "RMouth_x", "RMouth_y", "RMouth_z", " ", "LShoulder_x", "LShoulder_y", "LShoulder_z", " ",
        "RShoulder_x", "RShoulder_y", "RShoulder_z", " ", "LElbow_x", "LElbow_y", "LElbow_z", " ",
        "RElbow_x", "RElbow_y", "RElbow_z", " ", "LWrist_x", "LWrist_y", "LWrist_z", " ",
        "RWrist_x", "RWrist_y", "RWrist_z", " ", "LPinky_x", "LPinky_y", "LPinky_z", " ",
        "RPinky_x", "RPinky_y", "RPinky_z", " ", "LIndex_x", "LIndex_y", "LIndex_z", " ",
        "RIndex_x", "RIndex_y", "RIndex_z", " ", "LThumb_x", "LThumb_y", "LThumb_z", " ",
        "RThumb_x", "RThumb_y", "RThumb_z", " ", "LHip_x", "LHip_y", "LHip_z", " ",
        "RHip_x", "RHip_y", "RHip_z", " ", "LKnee_x", "LKnee_y", "LKnee_z", " ",
        "RKnee_x", "RKnee_y", "RKnee_z", " ", "LAnkle_x", "LAnkle_y", "LAnkle_z", " ",
        "RAnkle_x", "RAnkle_y", "RAnkle_z", " ", "LHeel_x", "LHeel_y", "LHeel_z", " ",
        "RHeel_x", "RHeel_y", "RHeel_z", " ", "LFootIdx_x", "LFootIdx_y", "LFootIdx_z", " ",
        "RFootIdx_x", "RFootIdx_y", "RFootIdx_z", " "
    ]

    # Plot the actual values and predictions in a horizontal layout
    plt.figure(figsize=(12, 16))
    plt.plot(actual_flat, range(len(y_labels)), label='Actual', color='blue', marker='o', markersize=6, linewidth=2, alpha=0.8)
    plt.plot(LSTMpredictions_flat, range(len(y_labels)), label='LSTM', color='red', linestyle='--', marker='x', markersize=5, linewidth=1.5, alpha=0.7)
    plt.plot(BiLSTMpredictions_flat, range(len(y_labels)), label='BiLSTM', color='green', linestyle='-.', marker='s', markersize=5, linewidth=1.5, alpha=0.7)
    plt.plot(TransformerPredictions_flat, range(len(y_labels)), label='Transformer', color='purple', linestyle=':', marker='^', markersize=5, linewidth=1.5, alpha=0.7)
    plt.plot(BERTpredictions_flat, range(len(y_labels)), label='BERT', color='orange', linestyle='-', marker='d', markersize=5, linewidth=1.5, alpha=0.7)

    plt.title(f'Comparison of Actual and Predicted Body Points for {stroke} Motion', fontsize=16, weight='bold')
    plt.ylabel('Body Keypoints', fontsize=12)
    plt.yticks(ticks=np.arange(len(y_labels)), labels=y_labels, fontsize=8)
    plt.xlabel('Values', fontsize=12)
    plt.legend(fontsize=10, loc='upper right')
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()

##################################################

def All_qualitative_quantitative_diff(
        actual, LSTMpredictions, BiLSTMpredictions, TransformerPredictions, BERTpredictions, stroke):
    import numpy as np
    import matplotlib.pyplot as plt

    # Flatten predictions and actual values to ensure they are one-dimensional arrays
    LSTMpredictions_flat = np.ravel(LSTMpredictions)
    BiLSTMpredictions_flat = np.ravel(BiLSTMpredictions)
    TransformerPredictions_flat = np.ravel(TransformerPredictions)
    BERTpredictions_flat = np.ravel(BERTpredictions)
    actual_flat = np.ravel(actual)

    # Define the x-axis labels (132 keypoints as given in the original code)
    x_labels = [
        "Nose_x", "Nose_y", "Nose_z", " ", "LEyeInner_x", "LEyeInner_y", "LEyeInner_z", " ",
        "LEye_x", "LEye_y", "LEye_z", " ", "LEyeOuter_x", "LEyeOuter_y", "LEyeOuter_z", " ",
        "REyeInner_x", "REyeInner_y", "REyeInner_z", " ", "REye_x", "REye_y", "REye_z", " ",
        "REyeOuter_x", "REyeOuter_y", "REyeOuter_z", " ", "LEar_x", "LEar_y", "LEar_z", " ",
        "REar_x", "REar_y", "REar_z", " ", "LMouth_x", "LMouth_y", "LMouth_z", " ",
        "RMouth_x", "RMouth_y", "RMouth_z", " ", "LShoulder_x", "LShoulder_y", "LShoulder_z", " ",
        "RShoulder_x", "RShoulder_y", "RShoulder_z", " ", "LElbow_x", "LElbow_y", "LElbow_z", " ",
        "RElbow_x", "RElbow_y", "RElbow_z", " ", "LWrist_x", "LWrist_y", "LWrist_z", " ",
        "RWrist_x", "RWrist_y", "RWrist_z", " ", "LPinky_x", "LPinky_y", "LPinky_z", " ",
        "RPinky_x", "RPinky_y", "RPinky_z", " ", "LIndex_x", "LIndex_y", "LIndex_z", " ",
        "RIndex_x", "RIndex_y", "RIndex_z", " ", "LThumb_x", "LThumb_y", "LThumb_z", " ",
        "RThumb_x", "RThumb_y", "RThumb_z", " ", "LHip_x", "LHip_y", "LHip_z", " ",
        "RHip_x", "RHip_y", "RHip_z", " ", "LKnee_x", "LKnee_y", "LKnee_z", " ",
        "RKnee_x", "RKnee_y", "RKnee_z", " ", "LAnkle_x", "LAnkle_y", "LAnkle_z", " ",
        "RAnkle_x", "RAnkle_y", "RAnkle_z", " ", "LHeel_x", "LHeel_y", "LHeel_z", " ",
        "RHeel_x", "RHeel_y", "RHeel_z", " ", "LFootIdx_x", "LFootIdx_y", "LFootIdx_z", " ",
        "RFootIdx_x", "RFootIdx_y", "RFootIdx_z", " "
    ]

    # Plot the actual values and predictions
    plt.figure(figsize=(18, 10))
    plt.plot(actual_flat, label='Actual', color='blue', marker='o', markersize=6, linewidth=2, alpha=0.8)
    plt.plot(LSTMpredictions_flat, label='LSTM', color='red', linestyle='--', marker='x', markersize=5, linewidth=1.5, alpha=0.7)
    plt.plot(BiLSTMpredictions_flat, label='BiLSTM', color='green', linestyle='-.', marker='s', markersize=5, linewidth=1.5, alpha=0.7)
    plt.plot(TransformerPredictions_flat, label='Transformer', color='purple', linestyle=':', marker='^', markersize=5, linewidth=1.5, alpha=0.7)
    plt.plot(BERTpredictions_flat, label='BERT', color='orange', linestyle='-', marker='d', markersize=5, linewidth=1.5, alpha=0.7)

    plt.title(f'Comparison of Actual and Predicted Body Points for {stroke} Motion', fontsize=16, weight='bold')
    plt.xlabel('Body Keypoints', fontsize=12)
    plt.xticks(ticks=np.arange(len(x_labels)), labels=x_labels,rotation=90,  fontsize=13)

    plt.ylabel('Values', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


In [5]:
from google.colab.patches import cv2_imshow
def pred_actual_frame( Posdataset, predictions_rescaled, y_test_rescaled):
    #Posdataset
    import cv2
    import pandas as pd
    import numpy as np
    dataset = Posdataset.drop(columns=['Unnamed: 0', 'Label'], axis=1)
    # Assuming 'newdataset' is your DataFrame
    columns_list = dataset.columns.tolist()

    preframes=predictions_rescaled

    # Reshape frames to 2D (if it is a 3D array)
    preframes = preframes.reshape(1, 132)

    # Now create the DataFrame
    PredictedFrame = pd.DataFrame(preframes, columns=columns_list)

    orgframes = y_test_rescaled
    orgframes = orgframes.reshape(1, 132)
    OrignalFrame = pd.DataFrame(orgframes, columns=columns_list)


    # Select only the columns that contain x, y, and z coordinates
    xyz_columns = [col for col in PredictedFrame.columns if '_x' in col or '_y' in col or '_z' in col]
    predicted_xyz = PredictedFrame[xyz_columns]
    original_xyz = OrignalFrame[xyz_columns]

    # Define connections between keypoints to draw the skeleton based on F1 to F32
    connections = [
        (0, 1), (1, 2), (2, 3),  # Left eye connections
        (0, 4), (4, 5), (5, 6),  # Right eye connections
        (0, 7), (0, 8),  # Nose to ears
        (9, 10),  # Mouth connection
        (11, 12),  # Shoulders
        (11, 13), (13, 15),  # Left arm (shoulder, elbow, wrist)
        (15, 17), (15, 19), (15, 21),  # Left hand (wrist to pinky, index, thumb)
        (12, 14), (14, 16),  # Right arm (shoulder, elbow, wrist)
        (16, 18), (16, 20), (16, 22),  # Right hand (wrist to pinky, index, thumb)
        (23, 24),  # Hips
        (11, 23), (12, 24),  # Shoulders to hips
        (23, 25), (25, 27), (27, 29), (27, 31),  # Left leg (hip, knee, ankle, heel, foot index)
        (24, 26), (26, 28), (28, 30), (28, 32),  # Right leg (hip, knee, ankle, heel, foot index)
    ]

    # Create a blank image (adjust size according to your coordinates)
    width, height = 640, 480  # Adjust size as needed

    # Loop through each row (frame) in the DataFrame to draw the skeleton for each frame
    for index in range(len(predicted_xyz)):
        # Clear the image for each frame
        blank_image = np.zeros((height, width, 3), dtype=np.uint8)

        # Extract (x, y) coordinates from the predicted row
        predicted_row = predicted_xyz.iloc[index]
        predicted_keypoints = []
        for i in range(0, len(predicted_row), 3):  # Skip every 3rd value because of the x, y, z format
            x = predicted_row[i] * width  # Assuming x is normalized, scale to image width
            y = predicted_row[i + 1] * height  # Assuming y is normalized, scale to image height
            predicted_keypoints.append((int(x), int(y)))

        # Extract (x, y) coordinates from the original row
        original_row = original_xyz.iloc[index]
        original_keypoints = []
        for i in range(0, len(original_row), 3):  # Skip every 3rd value because of the x, y, z format
            x = original_row[i] * width  # Assuming x is normalized, scale to image width
            y = original_row[i + 1] * height  # Assuming y is normalized, scale to image height
            original_keypoints.append((int(x), int(y)))

        # Draw circles at each keypoint for the predicted frame (color: green)
        for (x, y) in predicted_keypoints:
            cv2.circle(blank_image, (x, y), 5, (0, 255, 0), cv2.FILLED)

        # Draw lines to form the skeleton for the predicted frame (color: blue)
        for connection in connections:
            point1 = connection[0]
            point2 = connection[1]
            if point1 < len(predicted_keypoints) and point2 < len(predicted_keypoints):
                cv2.line(blank_image, predicted_keypoints[point1], predicted_keypoints[point2], (0, 255, 0), 2)

        # Draw circles at each keypoint for the original frame (color: red)
        for (x, y) in original_keypoints:
            cv2.circle(blank_image, (x, y), 5, (0, 0, 255), cv2.FILLED)

        # Draw lines to form the skeleton for the original frame (color: yellow)
        for connection in connections:
            point1 = connection[0]
            point2 = connection[1]
            if point1 < len(original_keypoints) and point2 < len(original_keypoints):
                cv2.line(blank_image, original_keypoints[point1], original_keypoints[point2], (0, 0, 255), 2)

        # Add text to indicate which color represents original and predicted
        cv2.putText(blank_image, 'Original: Pose Red', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        cv2.putText(blank_image, 'Predicted: Pose Green', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        #cv2.putText(blank_image, 'Original: Shot ' + str(Org_label[0]), (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        #cv2.putText(blank_image, 'Predicted: Shot ' + str(pred_label[0]), (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Show the skeleton for this frame
        #cv2.imshow('Skeleton Comparison', blank_image)
        '''
        cv2_imshow( blank_image)
        key = cv2.waitKey(50000) & 0xFF  # Adjust wait time to 30 milliseconds
        if key == ord('q'):
            print("Exiting on 'q' key press")
            break
        '''
    # Release resources
    cv2.destroyAllWindows()


    # Select only the columns that contain x, y, and z coordinates
    xyz_columns = [col for col in PredictedFrame.columns if '_x' in col or '_y' in col or '_z' in col]
    predicted_xyz = PredictedFrame[xyz_columns]
    original_xyz = OrignalFrame[xyz_columns]

    # Define connections between keypoints to draw the skeleton based on F1 to F32
    connections = [
        (0, 1), (1, 2), (2, 3),  # Left eye connections
        (0, 4), (4, 5), (5, 6),  # Right eye connections
        (0, 7), (0, 8),  # Nose to ears
        (9, 10),  # Mouth connection
        (11, 12),  # Shoulders
        (11, 13), (13, 15),  # Left arm (shoulder, elbow, wrist)
        (15, 17), (15, 19), (15, 21),  # Left hand (wrist to pinky, index, thumb)
        (12, 14), (14, 16),  # Right arm (shoulder, elbow, wrist)
        (16, 18), (16, 20), (16, 22),  # Right hand (wrist to pinky, index, thumb)
        (23, 24),  # Hips
        (11, 23), (12, 24),  # Shoulders to hips
        (23, 25), (25, 27), (27, 29), (27, 31),  # Left leg (hip, knee, ankle, heel, foot index)
        (24, 26), (26, 28), (28, 30), (28, 32),  # Right leg (hip, knee, ankle, heel, foot index)
    ]

    # Create a blank image (adjust size according to your coordinates)
    width, height = 640, 480  # Adjust size as needed

    # Loop through each row (frame) in the DataFrame to draw the skeleton for each frame
    for index in range(len(predicted_xyz)):
        # Clear images for each frame
        blank_image_predicted = np.zeros((height, width, 3), dtype=np.uint8)
        blank_image_original = np.zeros((height, width, 3), dtype=np.uint8)

        # Extract (x, y) coordinates from the predicted row
        predicted_row = predicted_xyz.iloc[index]
        predicted_keypoints = []
        for i in range(0, len(predicted_row), 3):  # Skip every 3rd value because of the x, y, z format
            x = predicted_row[i] * width  # Assuming x is normalized, scale to image width
            y = predicted_row[i + 1] * height  # Assuming y is normalized, scale to image height
            predicted_keypoints.append((int(x), int(y)))

        # Extract (x, y) coordinates from the original row
        original_row = original_xyz.iloc[index]
        original_keypoints = []
        for i in range(0, len(original_row), 3):  # Skip every 3rd value because of the x, y, z format
            x = original_row[i] * width  # Assuming x is normalized, scale to image width
            y = original_row[i + 1] * height  # Assuming y is normalized, scale to image height
            original_keypoints.append((int(x), int(y)))

        # Draw circles and lines for the predicted frame (color: green)
        for (x, y) in predicted_keypoints:
            cv2.circle(blank_image_predicted, (x, y), 5, (0, 255, 0), cv2.FILLED)
        for connection in connections:
            point1, point2 = connection
            if point1 < len(predicted_keypoints) and point2 < len(predicted_keypoints):
                cv2.line(blank_image_predicted, predicted_keypoints[point1], predicted_keypoints[point2], (0, 255, 0), 2)

        # Draw circles and lines for the original frame (color: red)
        for (x, y) in original_keypoints:
            cv2.circle(blank_image_original, (x, y), 5, (0, 0, 255), cv2.FILLED)
        for connection in connections:
            point1, point2 = connection
            if point1 < len(original_keypoints) and point2 < len(original_keypoints):
                cv2.line(blank_image_original, original_keypoints[point1], original_keypoints[point2], (0, 0, 255), 2)
        '''
        # Add text for frame labels
        cv2.putText(blank_image_original, 'Original', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        cv2.putText(blank_image_predicted, 'Predicted', (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Concatenate the two images side by side
        comparison_image = np.hstack((blank_image_original, blank_image_predicted))

        # Display the side-by-side comparison
        #cv2.imshow('Original vs Predicted Posture', comparison_image)
        cv2_imshow( comparison_image)
        # Wait for keypress or a delay between frames
        key = cv2.waitKey(50000) & 0xFF  # Adjust the delay as needed
        if key == ord('q'):
            print("Exiting on 'q' key press")
            break
        '''
    # Release resources
    cv2.destroyAllWindows()

In [7]:
from glob import glob
from google.colab import drive

import warnings
warnings.filterwarnings('ignore')

# Prepare the data for LSTM (create sequences)
def create_sequences(data, sequence_length, NextFrames):

    xs, ys = [], []
    for i in range(len(data) - sequence_length - NextFrames):  # We want to predict next 3 frames
        x = data[i:i+sequence_length]
        y = data[i+sequence_length:i+sequence_length+NextFrames]  # Predict 3 future frames
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

#mpPose = mp.solutions.pose
#pose = mpPose.Pose()
#mpDraw = mp.solutions.drawing_utils # For drawing keypoints
#points = mpPose.PoseLandmark # Landmarks
# Mount Google Drive
drive.mount('/content/drive')

base_path = '/content/drive/My Drive/Cricket_Enhancement/Drive/'  # Adjust as needed


# Load your datasets
Straight_Drive_df = pd.read_csv(base_path + "Straight_Drive.csv")
Sweep_df = pd.read_csv(base_path + "Sweep.csv")
Pull_df = pd.read_csv(base_path + "Pull.csv")
onDrive_df = pd.read_csv(base_path + "onDrive.csv")
Flick_df = pd.read_csv(base_path + "Flick.csv")
Cut_df = pd.read_csv(base_path + "Cut.csv")
Cover_Drive_df = pd.read_csv(base_path + "Cover_Drive.csv")
back_foot_punch_df = pd.read_csv(base_path + "back_foot_punch.csv")

# List of dataframes
dataframes = [
    #Straight_Drive_df,  Sweep_df, Pull_df, onDrive_df, Flick_df, Cut_df,   Cover_Drive_df,
    back_foot_punch_df
    ]
# Optional: List of names to identify each dataframe
names = [
    #"Straight_Drive", Sweep",   "Pull", "onDrive","Flick",  "Cut",     "Cover_Drive",
    "back_foot_punch"
    ]
#Now you can work with your DataFrames
#print(Cover_Drive_df.head())
results = []

#results_path = base_path + "original_BiLSTM_grid_search_results.csv"
#results_path = base_path + "original_Transformer_grid_search_results.csv"
results_path = base_path + "bert_grid_search_results.csv"
# Initialize an empty DataFrame to store the results
data_to_save = []
for i, df in enumerate(dataframes):
  data = []
  a=0
  # Load the dataset (replace 'Cover_Drive.csv' with your actual dataset file)
  #Posdataset =  Cover_Drive_df #pd.read_csv('Cover_Drive.csv')
  #print(f"\nFirst 3 rows of {names[i]}:")
  Posdataset=df
  stroke=names[i]
  # Drop unnecessary columns if they exist in each dataframe
  if 'Unnamed: 0' in df.columns and 'Label' in df.columns:
      Posdataset_numeric = df.drop(columns=['Unnamed: 0', 'Label'], axis=1)
  else:
      Posdataset_numeric = df  # If no unnecessary columns to drop
  # Extract all PoseNet columns (assuming columns are named as in your example)
  columns = ['F{}_{}'.format(i, axis) for i in range(33) for axis in ['x', 'y', 'z', 'vis']]
  data = Posdataset[columns].values

  # Normalize the data
  scaler = MinMaxScaler()
  data_scaled = scaler.fit_transform(data)
  print(data.shape)

  # Define a smaller sequence length (e.g., use 5 previous frames to predict the next 3 frames)
  sequence_length = 5 #10
  NextFrames=1 #5
  # Create sequences with the new sequence length
  #NextFrames=5 #3
  X, y = create_sequences(data_scaled, sequence_length, NextFrames)
  #print(math.floor(X.shape[0]*0.9) )
  predictedFrameNumber=math.floor(X.shape[0]*0.2)
  # Adjust the train/test split (e.g., 800 rows for training)
  train_size = math.floor(X.shape[0]*0.9) #800  # Adjust based on the number of frames you have

  X_train, y_train = X[:train_size], y[:train_size]
  X_test, y_test = X[train_size:], y[train_size:]

  from itertools import product
  import pandas as pd

  # Define the hyperparameter grid
  '''
  param_grid_LSTM = {
      'outputunit': [32, 64, 128],
      'learning_rate': [0.1, 0.01, 0.001],
       'batch_size': [16, 32, 64],
      }
  grid_combinations = list(product(*param_grid_LSTM.values()))
  param_names = list(param_grid_LSTM.keys())

  param_grid_BiLSTM = {
      'outputunit': [32, 64, 128],
      'learning_rate': [0.1,0.01, 0.001],
      'batch_size': [16, 32, 64],
      }
  grid_combinations = list(product(*param_grid_BiLSTM.values()))
  param_names = list(param_grid_BiLSTM.keys())

  param_grid_Transformer = {      'outputunit':[32, 64,128],      'MultiHead': [2, 4, 8],      'dropout': [0.1,0.2],

      }
  grid_combinations = list(product(*param_grid_Transformer.values()))
  param_names = list(param_grid_Transformer.keys())
  '''
  param_grid_BERT ={
      'num_transformer_blocks': [1,2,4],
      'outputunit': [1, 2, 4],
      'MultiHead': [2, 4, 8],
      'dropout': [0.1,0.2],
      }
  #Generate all parameter combinations
  grid_combinations = list(product(*param_grid_BERT.values()))
  param_names = list(param_grid_BERT.keys())

  # List to store the results
  results = []

  # Grid search
  for params in grid_combinations:
      param_dict = dict(zip(param_names, params))
      num_transformer_blocks = param_dict['num_transformer_blocks']
      outputunit = param_dict['outputunit']
      MultiHead = param_dict['MultiHead']
      dropout = param_dict['dropout']

      # Create sequences with the specified sequence_length and NextFrames
      X, y = create_sequences(data_scaled, sequence_length, NextFrames)
      train_size = math.floor(X.shape[0] * 0.9)
      X_train, y_train = X[:train_size], y[:train_size]
      X_test, y_test = X[train_size:], y[train_size:]
      n_features = data.shape[1]

      # Train and evaluate the BERT model
      #_,_,_, mae = lstm_model(NextFrames, data, X_train, y_train, X_test, y_test, train_size, outputunit, learning_rate, batch_size)
      #_,_,_,mae = Bilstm_model(NextFrames, data, X_train, y_train, X_test, y_test, train_size, outputunit, learning_rate, batch_size)

      _, _, _, mae = bert_model( NextFrames, sequence_length, n_features, X_train, y_train, X_test, y_test, train_size,
                                MultiHead, dropout, num_transformer_blocks=num_transformer_blocks )
      #_,_,_,mae = transformer_model(NextFrames, sequence_length, n_features, X_train, y_train, X_test, y_test,train_size,
      #outputunit, MultiHead,dropout)

      # Store the results
      result = {**param_dict, 'mae': mae}
      results.append(result)


  # Save results to a CSV file
  results_df = pd.DataFrame(results)
  #results_path = base_path + "bert_grid_search_results.csv"
  #results_path = base_path + "origianl_transformer_grid_search_results.csv"
  #results_df.to_csv(results_path, index=False)
  print(f"Stroke: {stroke}, outputunit: {outputunit}, MultiHead: {MultiHead}, num_transformer_blocks {num_transformer_blocks},  MAE: {mae}")
  results_df.to_csv(results_path, mode='a', header=False, index=False)
  print(f"Grid search completed. Results saved to {results_path}")


  '''
  # Print to check the shape of the data
  print(f"NextFrames: {NextFrames}, sequence_length: {sequence_length}, data.shape: {data.shape}")
  print(f"X shape: {X.shape}, y shape: {y.shape}")
  print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
  print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
  # Print to check the shape of the data
  print(f"data.shape: {data.shape}")
  print(f"data.shape[0]: {data.shape[0]}, data.shape[1]: {data.shape[1]}, data.shape[2]: {data.shape[1]}")
  '''
  # Ensure there are sequences in the test set
  if X_test.shape[0] == 0:
      raise ValueError("Test set (X_test) is empty. Adjust the data split or sequence length.")
  '''
  # LSTM Model
  lstm, actual, LSTMpredictions, mae = lstm_model(NextFrames, data, X_train, y_train, X_test, y_test, train_size)
  #draw_and_save_model(lstm, 'lstm_model.png')

  ModelName="LSTM"
  #mse, mse, r2, ModelName, stroke = model_evaluation(NextFrames, actual, LSTMpredictions, "LSTM", stroke, predictedFrameNumber)
  data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Actual", "Values": actual.tolist()})
  data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Predicted", "Values": LSTMpredictions.tolist()})
  print(f'LSTM model Mean Absolute Error for next {NextFrames} frames: {mae}')

  # BiLSTM Model
  ModelName="BiLSTM"
  bilstm, actual, BiLSTMpredictions, mae = Bilstm_model(NextFrames, data, X_train, y_train, X_test, y_test, train_size)
  #draw_and_save_model(bilstm, 'bilstm_model.png')

  #mse, mse, r2, ModelName, stroke = model_evaluation(NextFrames, actual, BiLSTMpredictions, "BiLSTM", stroke, predictedFrameNumber)
  #data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Actual", "Values": actual.tolist()})
  data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Predicted", "Values": BiLSTMpredictions.tolist()})
  print(f'BiLSTM model Mean Absolute Error for next {NextFrames} frames: {mae}')

  #LSTM_GridSearch(NextFrames, data, X_train, y_train, X_test, y_test, train_size)

  #BiLSTM_GridSearch(NextFrames, data, X_train, y_train, X_test, y_test, train_size)

  # Transformer Model
  ModelName="Transformer"
  n_features = data.shape[1]
  results_path = base_path + "transformer_grid_search_results3.csv"
  best_params, best_mae = Transformer_GridSearch(NextFrames, sequence_length, n_features, X_train, y_train, X_test, y_test, train_size, results_path)

  print(f"Best Parameters: {best_params}")
  print(f"Best MAE: {best_mae}")


  #draw_and_save_model(bertModel, 'bert_model_improved.png')
  #mse, mse, r2, ModelName, stroke = model_evaluation(NextFrames, actual, BERTpredictions, "BERT", stroke, predictedFrameNumber)
  #data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Actual", "Values": actual.tolist()})
  data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Predicted", "Values": BERTpredictions.tolist()})
  print(f'BERT model Mean Absolute Error for next {NextFrames} frames: {mae}')
 '''

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(850, 132)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 739ms/step
MultiHead: 2, dropout:0.1,  MAE: 0.05202040461794605
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 615ms/step
MultiHead: 2, dropout:0.2,  MAE: 0.05769045545163106
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step
MultiHead: 4, dropout:0.1,  MAE: 0.04962903547973045
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
MultiHead: 4, dropout:0.2,  MAE: 0.06342145930357519


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 708ms/step
MultiHead: 8, dropout:0.1,  MAE: 0.045992484788308585


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 515ms/step
MultiHead: 8, dropout:0.2,  MAE: 0.053211181060432156
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
MultiHead: 2, dropout:0.1,  MAE: 0.05932351475423093
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step
MultiHead: 2, dropout:0.2,  MAE: 0.06292410192559109
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 448ms/step
MultiHead: 4, dropout:0.1,  MAE: 0.05177633062587257
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step
MultiHead: 4, dropout:0.2,  MAE: 0.05430858168411675
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
MultiHead: 8, dropout:0.1,  MAE: 0.05013368856997833
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 545ms/step
MultiHead: 8, dropout:0.2,  MAE: 0.04796981750405517
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step
MultiHead: 2, dropout:0.1,  MAE: 0.04712866716476455
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 424ms/step
MultiHead: 2, dropout:0.2,  MAE: 0.055023933411812394
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 522ms/step
MultiHead: 4, dropout:0.1,  MAE: 0.05030511102442492
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 618ms/step
MultiHead: 4, dropout:0.2,  MAE: 0.0528

In [9]:
  transformermodel, actual, TransformerPredictions, mae = transformer_model(NextFrames, sequence_length, n_features, X_train, y_train, X_test, y_test, train_size)

  #draw_and_save_model(transformermodel, 'transformer_model.png')mae: 0.017732816221360977

  #mse, mse, r2, ModelName, stroke = model_evaluation(NextFrames, actual, TransformerPredictions, "Transformer", stroke, predictedFrameNumber)
  data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Actual", "Values": actual.tolist()})
  data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Predicted", "Values": TransformerPredictions.tolist()})
  print(f'Transformer model Mean Absolute Error for next {NextFrames} frames: {mae}')

  # BERT Model
  ModelName="BERT"
  n_features = data.shape[1]
  bertModel, actual, BERTpredictions, mae = bert_model(NextFrames, sequence_length, n_features, X_train, y_train, X_test, y_test, train_size)

  #draw_and_save_model(bertModel, 'bert_model_improved.png')
  #mse, mse, r2, ModelName, stroke = model_evaluation(NextFrames, actual, BERTpredictions, "BERT", stroke, predictedFrameNumber)
  #data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Actual", "Values": actual.tolist()})
  data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Predicted", "Values": BERTpredictions.tolist()})
  print(f'BERT model Mean Absolute Error for next {NextFrames} frames: {mae}')
  All_qualitative_quantitative_diff(actual,LSTMpredictions, BiLSTMpredictions,TransformerPredictions,BERTpredictions, stroke)

  # Save to CSV
  print("Results saved to 'model_predictions.csv' ")
  output_path = base_path + "model_predictions.csv"
  output_df = pd.DataFrame(data_to_save)
  output_df.to_csv(output_path, index=False)

  print(f"Results saved to: {output_path}")

  results_df = pd.DataFrame(results)
  print(results_df)


TypeError: transformer_model() missing 3 required positional arguments: 'outputunit', 'MultiHead', and 'dropout'

In [10]:
output_path = base_path + "model_predictions.csv"
output_df.to_csv(output_path, index=False)
print(f"Results saved to: {output_path}")

NameError: name 'output_df' is not defined

In [11]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# Wrap the Transformer model
def create_transformer_model(sequence_length, n_features, num_heads=4, key_dim=64, dense_units=64, dropout_rate=0.2):
    input_layer = Input(shape=(sequence_length, n_features))

    # Multi-Head Attention Layer
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(input_layer, input_layer)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Fully Connected Layers
    transformer_output = Dense(dense_units, activation='relu')(attention_output)
    transformer_output = Dropout(dropout_rate)(transformer_output)

    # Pooling to reduce dimensions to match NextFrames
    transformer_output = GlobalAveragePooling1D()(transformer_output)

    # Predict the next frames
    output_layer = Dense(NextFrames * n_features)(transformer_output)
    output_layer = Reshape((NextFrames, n_features))(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model

def Transformer_GridSearch(NextFrames, sequence_length, n_features, X_train, y_train, X_test, y_test, train_size, results_path):
    from itertools import product
    from sklearn.metrics import mean_absolute_error
    import pandas as pd

    # Define hyperparameter grid
    '''
    param_grid = {
        'num_heads': [2, 4, 8],
        'key_dim': [32, 64, 128],
        'dense_units': [32, 64, 128],
        'dropout_rate': [0.1, 0.2, 0.3],
        'batch_size': [16, 32, 64],
        'epochs': [10, 20, 30]
    }
    '''
        # Define hyperparameter grid
    param_grid = {
        'num_heads': [ 4, 6],
        'key_dim': [32, 64 ],
        'dense_units': [32, 64],
        'dropout_rate': [ 0.2, 0.3],
        'batch_size': [16, 32, 64],
        'epochs': [ 30]
    }

    best_mae = float('inf')
    best_params = None
    results = []  # List to store all combinations and their MAE

    # Iterate through all combinations of parameters
    for num_heads, key_dim, dense_units, dropout_rate, batch_size, epochs in product(
        param_grid['num_heads'],
        param_grid['key_dim'],
        param_grid['dense_units'],
        param_grid['dropout_rate'],
        param_grid['batch_size'],
        param_grid['epochs']
    ):
        print(f"Testing params: num_heads={num_heads}, key_dim={key_dim}, dense_units={dense_units}, "
              f"dropout_rate={dropout_rate}, batch_size={batch_size}, epochs={epochs}")

        # Create the model with the current parameters
        model = create_transformer_model(sequence_length, n_features, num_heads, key_dim, dense_units, dropout_rate)

        # Train the model
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

        # Evaluate on the test set
        predictions = model.predict(X_test)
        mae = mean_absolute_error(y_test.reshape(-1, n_features), predictions.reshape(-1, n_features))

        # Append the result to the list
        results.append({
            'num_heads': num_heads,
            'key_dim': key_dim,
            'dense_units': dense_units,
            'dropout_rate': dropout_rate,
            'batch_size': batch_size,
            'epochs': epochs,
            'MAE': mae
        })

        # Track the best parameters
        if mae < best_mae:
            best_mae = mae
            best_params = {
                'num_heads': num_heads,
                'key_dim': key_dim,
                'dense_units': dense_units,
                'dropout_rate': dropout_rate,
                'batch_size': batch_size,
                'epochs': epochs
            }
        print(f'dense_units: {dense_units}, dropout_rate: {dropout_rate}, batch_size: {batch_size}, MAE: {mae}')
    # Save the results to a CSV file
    results_df = pd.DataFrame(results)
    results_df.to_csv(results_path, index=False)
    print(f"Grid search results saved to: {results_path}")

    return best_params, best_mae


In [ ]:
def LSTM_GridSearch(NextFrames, data, X_train, y_train, X_test, y_test, train_size):

  # Define the hyperparameter grid
  param_grid = {
      'lstm_units': [64, 128, 256],
      'learning_rate': [0.001, 0.01],
      'batch_size': [16, 32,64],
      'epochs': [50, 100, 150]
  }

  # Generate all combinations of hyperparameters
  grid_combinations = list(itertools.product(*param_grid.values()))

  # Placeholder for results
  results = []

  # Perform grid search
  for idx, (lstm_units, learning_rate, batch_size, epochs) in enumerate(grid_combinations):
      print(f"Testing combination {idx + 1}/{len(grid_combinations)}: units={lstm_units}, lr={learning_rate}, batch={batch_size}, epochs={epochs}")

      lstm, actual, LSTMpredictions, mae =lstm_modelgrid(NextFrames, data, X_train, y_train, X_test, y_test, train_size, sequence_length, data.shape[1], lstm_units, learning_rate, batch_size, epochs)

      results.append({'lstm_units': lstm_units, 'learning_rate': learning_rate, 'batch_size': batch_size, 'epochs': epochs, 'mae': mae})
      ModelName="LSTM"
      #mse, mse, r2, ModelName, stroke = model_evaluation(NextFrames, actual, LSTMpredictions, "LSTM", stroke, predictedFrameNumber)
      #data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Actual", "Values": actual.tolist()})
      data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Predicted", "Values": LSTMpredictions.tolist()})
      print(f'LSTM model Mean Absolute Error for next {NextFrames} frames: {mae}')
      # Convert results to a DataFrame
      results_df = pd.DataFrame(results)


  csv_path = base_path + 'grid_search_results.csv'

  print(f"CSV file saved at: {csv_path}")

  results_df.to_csv(csv_path, index=False)

  # Display the best configuration
  best_result = results_df.loc[results_df['mae'].idxmin()]
  print("\nBest Configuration:")
  print(best_result)

  # Plot results
  plt.figure(figsize=(12, 6))
  plt.plot(range(len(results)), results_df['mae'], marker='o', label='MAE')
  plt.xticks(range(len(results)), [f"{i+1}" for i in range(len(results))], rotation=90)
  plt.xlabel('Parameter Combination Index')
  plt.ylabel('Mean Absolute Error (MAE)')
  plt.title('Grid Search Performance')
  plt.legend()
  plt.grid()
  plt.show()

#######################################
def BiLSTM_GridSearch(NextFrames, data, X_train, y_train, X_test, y_test, train_size):
  # Hyperparameter grid
  param_grid = {
      'units': [64, 128, 256],
      'batch_size': [16, 32, 64],
      'epochs': [50, 100, 150],
      'learning_rate': [0.01, 0.001, 0.0005]
  }

  # Generate all combinations of hyperparameters
  param_combinations = list(itertools.product(*param_grid.values()))

  # Track results
  results = []

  for params in param_combinations:
      units, batch_size, epochs, learning_rate = params
      mae = build_and_train_BiLSTM(units, batch_size, epochs, learning_rate, NextFrames, data, X_train, y_train, X_test, y_test, train_size)
      results.append({
          'units': units,
          'learning_rate': learning_rate,
          'batch_size': batch_size,
          'epochs': epochs,
          'mae': mae
      })
      ModelName="BiLSTM"
      #mse, mse, r2, ModelName, stroke = model_evaluation(NextFrames, actual, LSTMpredictions, "LSTM", stroke, predictedFrameNumber)
      #data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Actual", "Values": actual.tolist()})
      #data_to_save.append({"ModelName": ModelName, "Stroke": stroke, "Type": "Predicted", "Values": BiLSTMpredictions.tolist()})
      print(f'LSTM model Mean Absolute Error for next {NextFrames} frames: {mae}')
      # Convert results to a DataFrame
      results_df = pd.DataFrame(results)

  csv_path = base_path + 'BiLSTM_grid_search_results.csv'

  print(f"CSV file saved at: {csv_path}")
  results_df.to_csv(csv_path, index=False)

  # Display the best configuration
  best_result = results_df.loc[results_df['mae'].idxmin()]
  print("\nBest Configuration:", best_result)

  # Plotting MAE vs Hyperparameters
  plt.figure(figsize=(12, 6))
  for param in param_grid.keys():
      plt.plot(results_df[param], results_df['mae'], label=f'MAE vs {param}')
  plt.xlabel('Hyperparameter Values')
  plt.ylabel('Mean Absolute Error (MAE)')
  plt.title('Hyperparameter Tuning Results')
  plt.legend()
  plt.grid()
  plt.show()
###############################################



In [ ]:
# Define the LSTM model function
def lstm_modelgrid(NextFrames, data, X_train, y_train, X_test, y_test, train_size, sequence_length, n_features, lstm_units, learning_rate, batch_size, epochs):
    model = Sequential()
    model.add(LSTM(lstm_units, activation='relu', return_sequences=True, input_shape=(sequence_length, n_features)))
    model.add(LSTM(lstm_units, activation='relu'))
    model.add(Dense(NextFrames * n_features))
    model.add(Reshape((NextFrames, n_features)))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

    # Evaluate the model on the test set
    predictions_scaled = model.predict(X_test[:1])  # Predict only the first set of test data
    predictions = scaler.inverse_transform(predictions_scaled.reshape(NextFrames, -1))
    actual = data[train_size:train_size + NextFrames]
    mae = mean_absolute_error(actual, predictions)

    return model,actual,predictions, mae


# Function to create and train the BiLSTM model
def build_and_train_BiLSTM(units, batch_size, epochs, learning_rate, NextFrames, data, X_train, y_train, X_test, y_test, train_size):
    model = Sequential()
    model.add(Bidirectional(LSTM(units, activation='relu', return_sequences=True), input_shape=(X_train.shape[1], data.shape[1])))
    model.add(Bidirectional(LSTM(units, activation='relu')))
    model.add(Dense(NextFrames * data.shape[1]))
    model.add(Reshape((NextFrames, data.shape[1])))

    # Compile with adjustable learning rate
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    # Train the model
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)

    # Predict and evaluate
    predictions_scaled = model.predict(X_test[:1])
    predictions = scaler.inverse_transform(predictions_scaled.reshape(NextFrames, -1))
    actual = data[train_size:train_size+NextFrames]
    mae = mean_absolute_error(actual, predictions)

    return mae

############################################
def build_transformer_model(NextFrames, sequence_length, n_features, num_heads=4, key_dim=64, dense_units=64, dropout_rate=0.2):
    input_layer = Input(shape=(sequence_length, n_features))

    # Multi-Head Attention Layer
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(input_layer, input_layer)
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Fully Connected Layers
    transformer_output = Dense(dense_units, activation='relu')(attention_output)
    transformer_output = Dropout(dropout_rate)(transformer_output)

    # Pooling to reduce dimensions to match NextFrames
    transformer_output = GlobalAveragePooling1D()(transformer_output)

    # Predict the next frames
    output_layer = Dense(NextFrames * n_features)(transformer_output)
    output_layer = Reshape((NextFrames, n_features))(output_layer)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='mse')
    return model




In [ ]:
  '''
  # Compare predictions with actual data visually
  plt.figure(figsize=(12, 8))

  for i in range(NextFrames):  # For each of the 3 predicted frames
      plt.subplot(NextFrames, 1, i+1)
      plt.plot(predictions[i], label='Predicted Frame {}'.format(train_size + i + 1))
      plt.plot(actual[i], label='Actual Frame {}'.format(train_size + i + 1))
      plt.legend()
      plt.title(f'Frame {train_size + i + 1} Comparison')

  plt.tight_layout()
  plt.show()
  '''



In [ ]:
#Posdataset_numeric = Posdataset.drop(columns=['Unnamed: 0', 'Label'], axis=1)
#columns_list = Posdataset_numeric.columns.tolist()
#print(columns_list)

**The END**

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

dataset = Posdataset.drop(columns=['Unnamed: 0', 'Label'], axis=1)
# Assuming 'newdataset' is your DataFrame
columns_list = dataset.columns.tolist()

preframes = predictions_rescaled

# Reshape frames to 2D (if it is a 3D array)
preframes = preframes.reshape(1, 132)

# Now create the DataFrame
PredictedFrame = pd.DataFrame(preframes, columns=columns_list)

orgframes = y_test_rescaled
orgframes = orgframes.reshape(1, 132)
OrignalFrame = pd.DataFrame(orgframes, columns=columns_list)

# Load DataFrame with 33 joints (columns: 'x', 'y', 'z', 'visibility')
df1 = PredictedFrame  # Replace with your actual data
df2 = OrignalFrame  # Replace with your actual data

def extract_coordinates(df):
    joints = 33  # Number of joints
    x, y, z = [], [], []

    for i in range(joints):
        x.append(df1[f'F{i}_x'].values[0])  # Take the first row's x value
        y.append(df1[f'F{i}_y'].values[0])  # Take the first row's y value
        z.append(df1[f'F{i}_z'].values[0])  # Take the first row's z value

    return np.array(x), np.array(y), np.array(z)

# Extract coordinates for both poses
x1, y1, z1 = extract_coordinates(df1)
x2, y2, z2 = extract_coordinates(df2)

# Define connections between joints based on MediaPipe PoseNet model
POSE_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 7),  # Right arm
    (0, 4), (4, 5), (5, 6), (6, 8),  # Left arm
    (9, 10), (11, 12),               # Shoulders
    (11, 13), (13, 15), (15, 17),    # Left leg
    (12, 14), (14, 16), (16, 18),    # Right leg
    (11, 23), (12, 24),              # Hips to shoulders
    (23, 24), (23, 25), (24, 26),    # Hips and upper legs
    (25, 27), (27, 29), (29, 31),    # Left lower leg
    (26, 28), (28, 30), (30, 32)     # Right lower leg
]

# Function to plot the skeleton for a single pose in 3D
def plot_skeleton(x, y, z, connections, color='blue'):
    lines = []
    for joint1, joint2 in connections:
        lines.append(go.Scatter3d(
            x=[x[joint1], x[joint2]],
            y=[y[joint1], y[joint2]],
            z=[z[joint1], z[joint2]],
            mode='lines',
            line=dict(color=color, width=5)
        ))
    return lines

# Create the 3D plot with both poses
fig = go.Figure()

# Add skeleton for both poses (in red and blue)
fig.add_traces(plot_skeleton(x1, y1, z1, POSE_CONNECTIONS, color='red'))
fig.add_traces(plot_skeleton(x2, y2, z2, POSE_CONNECTIONS, color='blue'))

# Set the layout for the 3D plot with increased size
fig.update_layout(
    title='3D Pose Skeleton Visualization',
    scene=dict(
        xaxis=dict(title='X-axis'),
        yaxis=dict(title='Y-axis'),
        zaxis=dict(title='Z-axis', range=[-1, 1]),  # Adjust range as needed
        aspectmode='cube'
    ),
    width=1200,  # Increased width
    height=800,  # Increased height
    showlegend=False
)

# Display the plot
fig.show()


NameError: name 'Posdataset' is not defined